In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
import kagglehub
import os
# Download latest version
path = kagglehub.dataset_download("nitishabharathi/cert-insider-threat")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\pc\.cache\kagglehub\datasets\nitishabharathi\cert-insider-threat\versions\1


In [4]:
files = os.listdir(path)
print("Dataset Files:", files)
df = pd.read_csv(os.path.join(path, "email.csv"))
# Map Threat column to numerical (Yes -> 1, No -> 0)
df["Threat"] = np.random.choice(["Yes", "No"], size=len(df), p=[0.5, 0.5])
df["Threat"] = df["Threat"].map({"Yes": 1, "No": 0})
df = df.sample(n=500, random_state=42)
# Display first few rows
print(df.head(5))

Dataset Files: ['email.csv', 'psychometric.csv']
                               id                 date     user       pc  \
1375085  {X9O7-L3NQ83SH-7546ROCM}  09/08/2010 09:48:21  HWH0646  PC-8087   
1760687  {N4J5-H0DM81GR-0636AXZX}  11/18/2010 11:40:48  EMW0772  PC-6035   
489761   {T3T3-X0QU63SN-9148FQEX}  03/29/2010 14:40:35  HAD0246  PC-0926   
670295   {S1I7-L4JZ84KI-7061AWTL}  04/30/2010 10:57:44  ASB0796  PC-1130   
2611719  {C8B5-Q0BW49BI-1420XVWZ}  05/12/2011 11:45:23  HCW0419  PC-3772   

                                                        to   cc  \
1375085  Ursa.Britanney.Frederick@dtaa.com;Abel.Adam.Mo...  NaN   
1760687  Butler-Serina@yahoo.com;Vance-Quintessa@netzer...  NaN   
489761                                  TEB582@verizon.net  NaN   
670295   Tobias.Lucas.Morin@dtaa.com;Sonya.Samantha.Hah...  NaN   
2611719                         Herman.Ian.Abbott@dtaa.com  NaN   

                                  bcc                                from  \
1375085       

In [5]:
# Encoding categorical variables
categorical_cols = ['id', 'date', 'user', 'pc', 'to', 'cc', 'bcc', 'from', 'content']  # Adjust as needed
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Define features and target
X = df.drop(columns=['Threat'])  # Features
y = df['Threat']  # Target variable

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [7]:
# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [8]:
# Feature selection (using chi-square)
selector = SelectKBest(chi2, k=8)  # Choose best 8 features
X_train_selected = selector.fit_transform(X_train_resampled, y_train_resampled)
X_test_selected = selector.transform(X_test)

# Train Random Forest Classifier (Baseline)
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_selected, y_train_resampled)

# Predictions
y_pred_baseline = clf.predict(X_test_selected)

# Evaluate Baseline Model
baseline_results = {
    "Accuracy": accuracy_score(y_test, y_pred_baseline),
    "Precision": precision_score(y_test, y_pred_baseline, average='weighted'),
    "Recall": recall_score(y_test, y_pred_baseline, average='weighted'),
    "F1 Score": f1_score(y_test, y_pred_baseline, average='weighted'),
}

In [10]:
 #Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Train Random Forest Classifier (After Standardization)
clf_standardized = RandomForestClassifier(random_state=42)
clf_standardized.fit(X_train_scaled, y_train_resampled)

# Predictions after Standardization
y_pred_standardized = clf_standardized.predict(X_test_scaled)

# Evaluate Standardized Model
standardized_results = {
    "Accuracy": accuracy_score(y_test, y_pred_standardized),
    "Precision": precision_score(y_test, y_pred_standardized, average='weighted'),
    "Recall": recall_score(y_test, y_pred_standardized, average='weighted'),
    "F1 Score": f1_score(y_test, y_pred_standardized, average='weighted'),
}

In [11]:
# Apply PCA for dimensionality reduction
pca = PCA(n_components=5)  # Keep 5 principal components
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Train Random Forest Classifier (After PCA)
clf_pca = RandomForestClassifier(random_state=42)
clf_pca.fit(X_train_pca, y_train_resampled)

# Predictions after PCA
y_pred_pca = clf_pca.predict(X_test_pca)

# Evaluate PCA Model
pca_results = {
    "Accuracy": accuracy_score(y_test, y_pred_pca),
    "Precision": precision_score(y_test, y_pred_pca, average='weighted'),
    "Recall": recall_score(y_test, y_pred_pca, average='weighted'),
    "F1 Score": f1_score(y_test, y_pred_pca, average='weighted'),
}


In [ ]:
# Print results
print("Baseline Model Results:", baseline_results)
print("Standardized Model Results:", standardized_results)
print("PCA Model Results:", pca_results)